In [1]:
import dataclasses
import logging
import os
import sys
sys.path.append("..")
import math
from dataclasses import dataclass, field
from typing import Callable, Dict, List, Optional

import numpy as np
import torch
from torch.utils.data.sampler import RandomSampler
from torch.utils.data.dataloader import DataLoader

from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, EvalPrediction, GlueDataset, default_data_collator) 
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (HfArgumentParser, Trainer, TrainingArguments,
                          glue_compute_metrics, glue_output_modes,
                          glue_tasks_num_labels, set_seed)

from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers.data.processors.glue import MrpcProcessor, ColaProcessor, MnliProcessor, Sst2Processor, RteProcessor, WnliProcessor, QqpProcessor, QnliProcessor, StsbProcessor


from tqdm import tqdm, trange

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

In [4]:
model_args = ModelArguments(model_name_or_path = 'albert-base-v2')
data_args = DataTrainingArguments(task_name = 'MNLI', data_dir = '/home/nlp/data/glue_data/MNLI')
training_args = TrainingArguments(output_dir = '/home/nlp/experiments/meta/',
                                 do_eval = True,
                                 per_device_train_batch_size=64)


if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

# Set seed
set_seed(training_args.seed)

try:
    num_labels = glue_tasks_num_labels[data_args.task_name]
    output_mode = glue_output_modes[data_args.task_name]
except KeyError:
    raise ValueError("Task not found: %s" % (data_args.task_name))

In [5]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    finetuning_task=data_args.task_name,
    cache_dir=model_args.cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)


In [6]:
def build_compute_metrics_fn(task_name: str) -> Callable[[EvalPrediction], Dict]:
    def compute_metrics_fn(p: EvalPrediction) -> Dict:
        if output_mode == "classification":
            preds = np.argmax(p.predictions, axis=1)
        elif output_mode == "regression":
            preds = np.squeeze(p.predictions)
        return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

    return compute_metrics_fn

In [7]:
TrainingArguments

transformers.training_args.TrainingArguments

In [8]:
@dataclass
class Arguments(TrainingArguments):
    tasks = ['mrpc', 'cola', 'mnli', 'sst-2', 'rte', 'qqp', 'qnli', 'sts-b']
    target_task: str = field(default = 'mrpc')
    task_shared: bool = field(default = True)

    reload_model = None
    num_update_steps: int = field(default = 5)
    num_sample_tasks: int = field(default = 5)
    inner_learning_rate: float = field(default = 1e-3)
    glue_dir : str = field(default = None)
    max_len: int = field(default = 80)
    output_dir: str
    eval_steps = 10

In [9]:
args = Arguments(glue_dir='/home/nlp/data/glue_data', output_dir='/home/nlp/experiments/meta')

In [10]:
#args.tasks = ['mnli']

In [11]:
processor_dict = {
          'mrpc': MrpcProcessor,
          'cola': ColaProcessor,
          'mnli': MnliProcessor,
          'sst-2': Sst2Processor,
          'rte': RteProcessor,
          'wnli': WnliProcessor,
          'qqp': QqpProcessor,
          'qnli': QnliProcessor,
          'sts-b': StsbProcessor
        }
processors = [processor_dict[task]() for task in args.tasks]

In [12]:
GLUE_PATH = os.path.join('home', 'nlp', 'data', 'glue_data')
dataset_dict = {
          'mrpc': args.glue_dir+'/MRPC',
          'cola': args.glue_dir+'/CoLA',
          'mnli': args.glue_dir+'/MNLI',
          'sst-2': args.glue_dir+'/SST-2',
          'rte':  args.glue_dir+'/RTE',
          'wnli': args.glue_dir+'/WNLI',
          'qqp':  args.glue_dir+'/QQP',
          'qnli': args.glue_dir+'/QNLI',
          'sts-b': args.glue_dir+'/STS-B'
        }
data_dirs =  [dataset_dict[task] for task in args.tasks]

In [13]:
for i, task in enumerate(args.tasks):
        if task == args.target_task:
            target_task_id = i
            break

task_cluster_dict = {
      'mrpc': 0,
      'cola': 1,
      'mnli': 0,
      'sst-2': 1,
      'rte': 0,
      'wnli': 0,
      'qqp': 0,
      'qnli': 2,
      'sts-b': 3
    }
task_clusters = [task_cluster_dict[task] for task in args.tasks] if args.task_shared else None

In [14]:
label_lists = [processor.get_labels() for processor in processors]

In [15]:
task_clusters

[0, 1, 0, 1, 0, 0, 2, 3]

In [16]:
label_lists

[['0', '1'],
 ['0', '1'],
 ['contradiction', 'entailment', 'neutral'],
 ['0', '1'],
 ['entailment', 'not_entailment'],
 ['0', '1'],
 ['entailment', 'not_entailment'],
 [None]]

In [17]:
if not args.task_shared:
    num_labels = [len(label_list) for label_list in label_lists]
else:
    cluster_num_labels = {0: 3, 1: 2, 2: 2, 3: 1}
    num_labels = [cluster_num_labels[task_cluster] for task_cluster in task_clusters]

In [18]:
model.zero_grad()

In [19]:
num_labels

[3, 2, 3, 2, 3, 3, 2, 1]

In [20]:
 glue_output_modes[args.tasks[0]]

'classification'

In [21]:
data_dirs

['/home/nlp/data/glue_data/MRPC',
 '/home/nlp/data/glue_data/CoLA',
 '/home/nlp/data/glue_data/MNLI',
 '/home/nlp/data/glue_data/SST-2',
 '/home/nlp/data/glue_data/RTE',
 '/home/nlp/data/glue_data/QQP',
 '/home/nlp/data/glue_data/QNLI',
 '/home/nlp/data/glue_data/STS-B']

In [22]:
train_dataset_list, eval_dataset_list = [], []
for task, data_dir in tqdm(zip(args.tasks, data_dirs)):
    data_args.task_name = task
    data_args.data_dir = data_dir
    train_dataset_list.append(GlueDataset(data_args, tokenizer))
    eval_dataset_list.append(GlueDataset(data_args, tokenizer, mode = "dev"))

8it [00:23,  2.97s/it]


In [23]:
from collections import defaultdict, OrderedDict


class MetaDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.args =  self.dataset.args
        self.processor = self.dataset.processor
        self.features = self.dataset.features
        self.label_list = self.dataset.label_list
        self.output_mode = self.dataset.output_mode
        self.indices_mapping = self._get_indices_mapping()
        self.num_labels = len(self.indices_mapping.keys())
        self.min_len = self.get_len()
        #self.data = self.form_data()
        self.data = self.get_tensorized_data()
    
    def get_len(self):
        min_len = float("inf")
        for values in self.indices_mapping.values():
            min_len = min(len(values), min_len)
        return min_len
        
    def __len__(self):
        return self.min_len
    
    def _get_indices_mapping(self):
        indices_mapping = {}
        for idx, data in enumerate(self.dataset):
            indices_mapping.update({idx: data.label})
        
        temp_mapping = defaultdict(list)
        for key, value in sorted(indices_mapping.items()):
            temp_mapping[value].append(key)
        
        indices_mapping = temp_mapping
        del temp_mapping
        return indices_mapping
        
    def __getitem__(self, idx):
        #res = []
        # res = OrderedDict()
        #for label in self.indices_mapping.keys():
            # res[label] = self.features[self.indices_mapping[label][idx]]
        #    res.append(self.features[self.indices_mapping[label][idx]])
        
        return self.data[idx]
    
    def get_tensorized_data(self):
        tensorized_data = []
        dtype = torch.long
        #for i, data in tqdm(enumerate(self.data)):
        for idx in trange(self.min_len):
            res = []
            for label in range(self.num_labels):
                data = self.features[self.indices_mapping[label][idx]]
                res.append({
                    'input_ids': torch.tensor(data.input_ids, dtype=dtype),
                    'attention_mask': torch.tensor(data.attention_mask, dtype=dtype),
                    'token_type_ids': torch.tensor(data.token_type_ids, dtype=dtype),
                    'labels': torch.tensor(data.label, dtype=dtype)
                })
            tensorized_data.append(res)
        return tensorized_data
    


In [24]:
meta_dataset = MetaDataset(train_dataset_list[2])

100%|██████████| 130899/130899 [00:14<00:00, 9283.70it/s] 


In [25]:
meta_dataset[1000]

[{'input_ids': tensor([    2,    28,  3820,    15,  1033,   877,  1437,    22,    38,   164,
            140,    20,    14,   877,    22,    18,   683,   584,    13,     5,
           9700, 16514,     6,   163,    14,  1788,     9,     3,  1033,   877,
           3049,   683,    17,  5339,    37,    14,   799,     3,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [26]:
sampler = RandomSampler(meta_dataset)

In [49]:
dataloader = DataLoader(meta_dataset,
            batch_size=8,
            sampler=sampler,
            #collate_fn=default_data_collator,
            drop_last=True)

In [28]:
loss = model(**next(iter(dataloader))[0])[0]

In [29]:
torch.tensor([1.2166], requires_grad=True).cuda() + torch.tensor(0.0).cuda()

tensor([1.2166], device='cuda:0', grad_fn=<AddBackward0>)

In [50]:
batch = next(iter(dataloader))

In [51]:
batch[0]['input_ids'].shape

torch.Size([8, 128])

In [214]:
params = gradient_update_parameters(model, loss[0])

In [215]:
params

OrderedDict()

In [226]:
loss[0]+torch.tensor(0.)

tensor(0.6255, grad_fn=<AddBackward0>)

In [227]:
loss[0]

tensor(0.6255, grad_fn=<NllLossBackward>)

In [33]:
train_sampler_list = []
for dataset in train_dataset_list:
    train_sampler_list.append(RandomSampler(dataset))

In [35]:
train_dataloader_list, eval_dataloader_list = [], []
data_collator = default_data_collator

for train_dataset, eval_dataset, sampler in \
    tqdm(zip(train_dataset_list, eval_dataset_list, train_sampler_list)):
    
    train_dataloader_list.append(DataLoader(train_dataset,
            batch_size=training_args.train_batch_size,
            sampler=sampler,
            collate_fn=data_collator,
            drop_last=True))
    
    eval_dataloader_list.append(DataLoader(eval_dataset,
            batch_size=training_args.train_batch_size,
            sampler=sampler,
            collate_fn=data_collator,
            drop_last=True))

1it [00:00, 106.08it/s]


In [129]:
train_examples = [processor.get_train_examples(data_dir) for processor, data_dir in tqdm(zip(processors, data_dirs))]

8it [00:06,  1.15it/s]


In [220]:
training_args.num_epochs = 1
training_args.per_device_train_batch_size = 1
args.num_update_steps = 0

In [221]:
train_steps_per_task = [ math.floor((len(train_example)/training_args.per_device_train_batch_size)/(args.num_update_steps+1)) for train_example in train_examples]
total_steps = sum(train_steps_per_task) * training_args.num_train_epochs
print(f'Total steps: {total_steps}')

Total steps: 2847303.0


In [222]:
train_steps_per_task

[3668, 8551, 392702, 67349, 2490, 363849, 104743, 5749]

In [48]:
training_args.per_device_train_batch_size

64

In [49]:
label_lists

[['contradiction', 'entailment', 'neutral']]

In [50]:
args.tasks

['mnli']

In [51]:
train_steps_per_task

[1022]

In [52]:
t_total = int(len(train_dataloader_list) // training_args.gradient_accumulation_steps * training_args.num_train_epochs)
num_train_epochs = training_args.num_train_epochs

In [53]:
train_dataloaders_iters = [iter(train_dataloader) for train_dataloader in train_dataloader_list]

extra_ids = []
for t_id in range(len(args.tasks)):
    extra_ids += [t_id] * train_steps_per_task[t_id]  #math.ceil(len(train_examples[t_id]))
extra_ids = np.random.choice(extra_ids, len(extra_ids), replace=False) 

In [55]:
extra_ids

array([0, 0, 0, ..., 0, 0, 0])

In [79]:
len(train_dataset_list)

1

In [82]:
indices_train_dataset_list = []
for dataset in train_dataset_list:
    cur_len = len(dataset)
    indices = np.arange(cur_len)
    np.random.shuffle(indices)
    indices_train_dataset_list.append(indices)

In [83]:
indices_train_dataset_list

[array([311245,  45402, 358631, ..., 152286, 137138, 179078])]

In [76]:
indices = np.arange(len(train_dataset_list[0]))

In [77]:
np.random.shuffle(indices)

In [78]:
indices

array([ 80976,  36001, 260520, ..., 292077, 102911, 114843])

In [ ]:
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

In [31]:
 def empty_memory():
    import gc
    gc.collect()
    torch.cuda.empty_cache()

In [26]:
class MetaTrainer(Trainer):
    def __init__(self, 
                 model, 
                 args, 
                 train_dataloader_list,
                 eval_dataloader_list,
                 compute_metrics,
                 prediction_loss_only = False):
        
        self.model = model.to(args.device)
        self.args = args
        self.compute_metrics_list = [compute_metrics(task) for task in self.args.tasks]
        self.train_dataloader_list = train_dataloader_list
        self.eval_dataloader_list = eval_dataloader_list
        self.data_collator = DefaultDataCollator()
        self.prediction_loss_only = prediction_loss_only
        self.eval_results = {}
        self._setup_wandb()
        
    def update_model_params(self, model, fast_params):
        for idx, param in enumerate(model.parameters()):
            param.data = fast_params[idx]
        return model

    def train(self):
        model = self.model
        
        self.global_step = 0
        nb_tr_steps = 0
        tr_loss = 0
        optimizer, scheduler = self.get_optimizers(int(len(self.train_dataloader_list[0]) \
        // self.args.gradient_accumulation_steps * self.args.num_train_epochs))
        
        model.zero_grad()
        
        for _ in trange(int(training_args.num_train_epochs), desc='Epoch'):
            model.train()
            tr_loss = 0
            nb_tr_examples = 0 

            train_dataloaders_iters = [iter(train_dataloader) for train_dataloader in train_dataloader_list]

            extra_ids = []
            for t_id in range(len(args.tasks)):
                extra_ids += [t_id] * train_steps_per_task[t_id]  #math.ceil(len(train_examples[t_id]))
            extra_ids = np.random.choice(extra_ids, len(extra_ids), replace=False) 

            meta_loss = 0

            for i, task_id in tqdm(enumerate(extra_ids), desc='Task IDs'):
                for update_step in range(args.num_update_steps+1):

                    try:
                        inputs = next(train_dataloaders_iters[task_id])
                    except StopIteration:
                        break

                    for k, v in inputs.items():
                        inputs[k] = v.to(args.device)
                        if not isinstance(inputs['labels'], torch.cuda.LongTensor):
                            inputs['labels'] = inputs['labels'].long()

                    if update_step == args.num_update_steps:
                        if update_step == 0:
                            raise ValueError('update_step cannot be 0!')

                        for param, f_param in zip(model.parameters(), fast_params):
                            if not param.requires_grad:
                                continue
                            cur_grad = (param - f_param)/update_step/args.inner_learning_rate
                            if param.grad is None:
                                param.grad = torch.zeros(cur_grad.size()).cuda()
                                param.grad.add_(cur_grad/inputs['labels'].size(0))

                    elif update_step == 0:
                        loss = model(**inputs)[0]
                        grad = torch.autograd.grad(loss, model.parameters(), allow_unused=True)
                        fast_params = list(map(lambda p: p[1] - args.inner_learning_rate * p[0] if p[0] is not None else p[1], zip(grad, model.parameters())))

                    elif update_step < args.num_update_steps:
                        model = self.update_model_params(model, fast_params)
                        loss = model(**inputs)[0]
                        grad = torch.autograd.grad(loss, fast_params, allow_unused=True)
                        fast_params = list(map(lambda p: p[1] - args.inner_learning_rate * p[0] if p[0] is not None else p[1], zip(grad, fast_params)))

                if i % args.num_sample_tasks == (args.num_sample_tasks-1):
                    optimizer.step()
                    optimizer.zero_grad()
                    meta_loss = 0

                self.global_step += 1
        
                if self.global_step % args.eval_steps == 0:
                    for idx, eval_dataloader in enumerate(self.eval_dataloader_list):
                        self.compute_metrics = self.compute_metrics_list[idx]
                        result = self.evaluate(eval_dataloader.dataset)
                        
                        for key, value in result.items():
                            logger.info("%s  %s = %s",args.tasks[idx], key, value)
                        

In [27]:
trainer = MetaTrainer(model, args, train_dataloader_list,
                     eval_dataloader_list, build_compute_metrics_fn)

In [29]:
trainer.train()

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
Task IDs: 0it [00:00, ?it/s]
Task IDs: 1it [00:03,  3.75s/it]
Task IDs: 2it [00:06,  3.31s/it]
Task IDs: 3it [00:08,  3.00s/it]
Task IDs: 4it [00:10,  2.79s/it]
Task IDs: 5it [00:12,  2.64s/it]
Task IDs: 6it [00:15,  2.55s/it]
Task IDs: 7it [00:17,  2.47s/it]
Task IDs: 8it [00:19,  2.42s/it]
Task IDs: 9it [00:22,  2.39s/it]

/home/nlp/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



{"eval_loss": 1.1688626500276418, "eval_pearson": -0.07593542849008117, "eval_spearmanr": -0.07026298108615246, "eval_corr": -0.07309920478811682, "step": 10}



{"eval_loss": 1.7732348694945828, "eval_pearson": 0.047892375448013634, "eval_spearmanr": 0.04789237544801361, "eval_corr": 0.04789237544801363, "step": 10}



{"eval_loss": 1.125631565185634, "eval_pearson": 0.03112561928614942, "eval_spearmanr": 0.031110041573893667, "eval_corr": 0.031117830430021545, "step": 10}


...


{"eval_loss": 1.2278344829230043, "eval_pearson": -0.01632747615200033, "eval_spearmanr": -0.0161722522877514, "eval_corr": -0.016249864219875863, "step": 20}



{"eval_loss": 1.1502182028510355, "eval_pearson": 0.005972333635109359, "eval_spearmanr": 0.009997267686546754, "eval_corr": 0.007984800660828056, "step": 20}



{"eval_loss": 0.8318254517184364, "eval_pearson": 0.0029989208045044153, "eval_spearmanr": 0.0029989208045043867, "eval_corr": 0.002998920804504401, "step": 20}


Task IDs: 19it [03:38, 11.49s/it]
Epoch:   0%|          | 0/3 [03:38<?, ?it/s]

KeyboardInterrupt: 

In [35]:
data_args

GlueDataTrainingArguments(task_name='sts-b', data_dir='/home/nlp/data/glue_data/STS-B', max_seq_length=128, overwrite_cache=False)

In [36]:
label_lists

[['0', '1'],
 ['0', '1'],
 ['contradiction', 'entailment', 'neutral'],
 ['0', '1'],
 ['entailment', 'not_entailment'],
 ['0', '1'],
 ['entailment', 'not_entailment'],
 [None]]

In [ ]:
# del trainer
# import gc
# gc.collect()
# torch.cuda.empty_cache()